In [45]:
from torchvision.datasets import ImageFolder
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transforms
import torch.nn.functional as F
from sklearn.preprocessing import OneHotEncoder
import PIL
from PIL import Image
import PIL.Image as pilimg
import torch
from torch.utils.data.sampler import SubsetRandomSampler
import torch.nn as nn
import torch.optim.lr_scheduler as lr_scheduler
import torch.optim as optim
import math
import pandas as pd
import numpy as np


import time
from model import Res2Net
from dataloader import data_generator_test

df = pd.read_csv('./result2.csv')
df.columns=['num','path','gt1','gt2','pred','top1','top2','top3','top4','top5']


In [46]:
def label_category(label):
    category_11 = 0 # ambulance etc
    category_4 = 1  #ambulance, indoor, outdoor, highway, transportation
    category_2 = 0 # not ambulance, ambulance, 
    
    ## ambulance 
    if label == 'ambulance' :
        category_4 = 0
        category_2 = 1
        return 0,0,1
    ## accomdation or buildings
    elif label == "accomodation" or  label =='village' or  label == 'hotel_room' or  label =='hallway' or label == 'elevator'or label == 'lobby'or label == 'office'  or  label =='library_indoor' :
        
        category_11 = 1
        return 1,1,0
    ## shopping mall
    elif label == "department_store" or label == 'shoppingmall_indoor' or  label =='supermarket'or  label =='restaurant':  
        
        category_11 = 2
        return 2,1,0
    ## construction, factory        
    elif label == 'engine_room' or label == 'factory' or label == 'construc_facto'  or label == 'auto_factory'or label == 'repairshop': 
        category_11 = 3
        return 3,1,0
    
    elif label == "construction_site" or label == 'excavation'or label == 'factory_outdoor': 
        category_4 = 3
        category_11 = 3        
        return 3,2,0
    ## 여가시설  #실내
    elif label == "amusement_arcade" or label == 'aquarium' or label == "art_gallery" or label =='auditorium' or label =='bookstore' : 
        category_11 = 4        
        return 4,1,0
    elif label ==  'movie_theater' or label == 'museum' or  label == "swimming_pool" or  label =='museum-indoor' or label=='swim_indoor':
        category_11 = 4        
        return 4,1,0
    # 실외
    elif label == 'amusement_park' or  label == "water_park" or label == "park" or   label =='museum-outdoor' or  label =='movie_theater-indoor' or label=='swim_outdoor' or label=='playground':
        category_4 = 2
        category_11 = 4    
        return 4,2,0
    # home
    elif label == "bedroom" or  label =='dining_room' or  label =='kitchen'or  label =='living_room'or  label =='restroom':           
        category_11 = 5
        return 5,1,0
    # hospital
    elif label == "dentaloffice" or label == 'hospitalroom' or  label =='hospital_room'or   label =='operating_room' or  label == 'nursing_home' :            
        
        category_11 = 6
        return 6,1,0
    #nature
    elif label == "beach" or label == 'broadleaf' or label == 'forest_path' or label == 'lagoon' or label == 'mountain_snowy'or label == 'forest_road' or label == 'lake_natural'or label == 'ocean'or label == 'raft' or label == 'river' or label == 'mountain_path' or label == 'lake-natural':
        category_11 = 7
        category_4 = 2
        return 7,2,0
    # school
    elif label == "classroom" or label == 'kindergarden_classroom' or label ==  'lecture_room':
        
        category_11 = 8
        return 8,1,0
    elif label == "campus" :
        return 8,2,0
    elif label == "athletic_field" or label == 'football_field' or label ==  'golf_course' or label ==  "gymnasium" or label == 'soccer_field' or label == 'stadium'or label == "baseball" or label == 'football' or label == 'soccer':
        category_11 = 9
        category_4 = 2
        return 9,2,0
    elif label == "gymnasium" or label == 'stadium':
        category_11 = 9
        category_4 = 1
        return 9,1,0
    elif label == "crosswalk" or label ==  'street' or label =='residential_neighborhood' or label ==  'street_alley' :
        category_11 = 10
        category_4 = 3
        return 10,2,0
    elif label == 'parking_lot' or label == 'parking-garage-outdoor' :
        category_11 = 11
        category_4 = 2
        return 11,2,0
    elif label == 'parking_garage' or label ==  'parking_garage_indoor':
        category_11 = 11
        category_4 = 1
        return 11,1,0
    elif label == 'highway':
        
        category_11 = 12
        category_4 = 4
        return 12,3,0
    elif label == 'bus' or  label == 'subway'or label == 'subway_station' or label == 'car_indoor':
        
        category_11 = 13
        category_4 = 2
        return 13,4,0
    else : 
        return 0,0,0
    
#     return category_11, category_4, category_2
        

In [47]:
df

,num,path,gt1,gt2,pred,top1,top2,top3,top4,top5
0,0,./data/val/hallway/capture_3_58.jpg,3,hallway,0,ambulance,street,office,operating_room,parking_garage_indoor
1,1,./data/val/hallway/add_7.jpg,3,hallway,1,bedroom,restaurant,street,hospital_room,office
2,2,./data/val/hallway/capture_3_55.jpg,3,hallway,12,restroom,bedroom,elevator,street,restaurant
3,3,./data/val/hallway/capture_3_63.jpg,3,hallway,12,restroom,bedroom,living_room,restaurant,kitchen
4,4,./data/val/hallway/capture_3_60.jpg,3,hallway,12,restroom,restaurant,kitchen,hospital_room,bedroom
...,...,...,...,...,...,...,...,...,...,...
195,195,./data/val/bedroom/capture_2_89.jpg,1,bedroom,0,ambulance,restaurant,bedroom,street,hospital_room
196,196,./data/val/bedroom/capture_2_33.jpg,1,bedroom,2,elevator,restroom,bedroom,street,living_room
197,197,./data/val/bedroom/capture_2_76.jpg,1,bedroom,12,restroom,bedroom,living_room,restaurant,kitchen
198,198,./data/val/bedroom/capture_2_94.jpg,1,bedroom,0,ambulance,bedroom,hospital_room,street,restaurant


In [48]:
gt=df['gt2'].tolist()
pred=df['top1'].tolist()
print('number of prediction : %i' % len(pred))
print('number of truth : %i' % len(gt)) 

print(pred[0])
print(gt[0])
# print(label_category(gt[0]))
# print(label_category(gt[0]))

number of prediction : 200
number of truth : 200
ambulance
hallway


In [49]:
score_list_1 = []
score_list_2 = []
score_list_3 = []
score_1=0
score_2=0
score_3=0
count_list=[]
count=0
for i in range(len(pred)):
    gt_label_place, gt_label_ind_outd, gt_label_ambul = label_category(gt[i])
    if  gt_label_place == 12 :
        count=count+1

         
        
    pred_label_place, pred_label_ind_outd, pred_label_ambul = label_category(pred[i])

#     print(label_category(pred[i]))
    if gt_label_place == pred_label_place :
        score_1=1
    else :
        score_1=0
    
    if gt_label_ind_outd == pred_label_ind_outd :
        score_2=1
    else :
        score_2=0
        
    if gt_label_ambul == pred_label_ambul :
        score_3=1
    else :
        score_3=0    
#     print(i, score_1, score_2, score_3)
    score_list_1.append(score_1)
    score_list_2.append(score_2)
    score_list_3.append(score_3)
# print(sum(score_list_1))
# print(sum(score_list_2))
# print(sum(score_list_3))
place_acc = sum(score_list_1) / len(score_list_1)
in_out_acc =  sum(score_list_2) / len(score_list_2)
ambul_acc =  sum(score_list_3) / len(score_list_3)
print('place accuracy = %.6f' % place_acc)
print('indoor_outdoor accuracy = %.6f' % in_out_acc)
print('ambulance accuracy = %.6f' % ambul_acc)

print(count/100*5000     )

place accuracy = 0.290000
indoor_outdoor accuracy = 0.625000
ambulance accuracy = 0.715000
0.0


In [14]:
score_list_3

[0,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 0,
 0,
 1,
 0,
 1,
 1,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 0,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 0,
 1,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 0,
 1]

## 

In [6]:
# def label_category(label):
#     category = 0
#     if label == "highway" :
#         category = 1
#     elif label == "street" or label == 'residential_neighborhood' :
#         category = 2
#     else : 
#         category = 3
#     return category
# # #     category = 0 
# #     if label == "accomodation" or label == 'village' or label =='residential_neighborhood' or label=='hotel_room' or label =='hallway' or label == 'elevator'or label == 'office': 
# #         return  1  ##accomd
# #     elif label == "department_store" or label =='shoppingmall-indoor' or label == 'supermarket': 
# #         return  2   ## mall
# #     elif label == "construction_site" or label =='engine_room' or label == 'factory' or label =='construc_facto' or label == 'excavation': 
# #         return  3  ## construction, factory
# #     elif label == 'auto_factory':
# #         return 3
# #     elif label == "amusement_arcade" or label =='amusement_park' or label == 'aquarium' or label == "art_gallery" or label =='auditorium' : 
        
# #         return  4 ##여가시설
# #     elif label == 'bookstore' or label == "library" or label =='movie_theater' or label == 'museum' or label == "swimming_pool" or label == "water_park" : 
# #         return  4 ##여가시설
# #     elif label == 'library-indoor' or label =='library-outdoor' or label=='museum-indoor'or label=='museum-outdoor':
        
# #         return  4 ##여가시설        
# #     elif label == 'library-indoor' or label =='movie_theater-indoor' or label=='swim-outdoor'or label=='swim-indoor':
# #         return  4 ##여가시설        
# #     elif label == "bedroom" or label =='dining_room' or label == 'kitchen'or label == 'living_room':            
# #         return  5 ##집
# #     elif label == "dentaloffice" or label =='hospitalroom' or label == 'hospital_room'or label == 'operating_room' or label == 'nursing_home' :            
# #         return  6
# #     elif label == "beach" or label == 'broadleaf' or label =='forest_path' or label =='lagoon' or label =='mountain_snowy'or label =='forest_road':
# #         return  7
# #     elif label =='lake_natural'or label =='ocean'or label =='raft' or label =='river' or label == 'mountain_path' or label=='lake-natural':
# #         return  7
# #     elif label == "classroom" or label =='kindergarden_classroom' or label == 'lecture_room':
# #         return  8
# #     elif label == "athletic_field" or label =='football_field' or label == 'golf_course' or label == "gymnasium" or label =='soccer_field' or label == 'stadium':
# #         return  9
# #     elif label == "baseball" or label == 'football' or label =='soccer' :
# #         return  9
# #     elif label == "crosswalk" or label =='street' :
# #         return  10
# #     elif label == "campus" or label =='parking_garage' or label =='parking_lot' or label=='parking-garage-outdoor' or label=='parking-garage-indoor':
        
        
# #         return  11
# #     elif label == "campus" or label =='parking_garage' or label =='parking_lot' or label=='parking-garage-outdoor' or label=='parking_garage-indoor' or label=='ambulance' :
# #         return 11
